In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import all neccessary libraries 

In [ ]:
import numpy as np 
import pandas as pd
import missingno as msno # for finding missing values in the data
import matplotlib.pyplot as plt
import seaborn as sns 

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from IPython.display import display

In [ ]:
data=pd.read_csv("/kaggle/input/fifa-20-complete-player-dataset/players_20.csv")
uselesscolumn = ['dob','sofifa_id','player_url','long_name','body_type','real_face','nation_position','loaned_from','nation_jersey_number',]
data = data.drop(uselesscolumn,axis = 1)
data.head()

In [ ]:
columns = ['ls','st','rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm',
           'rcm','rm','lwb','ldm', 'cdm','rdm','rwb','lb','lcb','cb','rcb','rb']
for col in columns:
    data[col] = data[col].str.split('+').str[0]
    
data[columns] = data[columns].fillna(0)

#changig the dtype to 'int'
data[columns] = data[columns].astype('int')

for col in columns:
    data[col] = data[col].replace(0,(data[col].mean()))
data[columns]

attributes = ['dribbling','defending','physic','passing','shooting','pace']
for att in attributes:
    data[att] = data[att].fillna(data[att].mean())

***Top 10 Attributes Generator***

In [ ]:
def f(x):
    display(x)
    return x

C = data.columns
P = interactive(f, x=widgets.Dropdown(options=C,value='overall',description='Attribute:',disabled=False))
P

In [ ]:
Att = P.result
Gen = data.sort_values(by = Att , ascending = False)[['short_name','club',Att,'overall']].head(10)
print("The top 10 players are: ")
print(Gen)

fig,ax = plt.subplots(figsize = (12,6))
sns.barplot(Gen['short_name'],Gen[Att], color = 'blue',palette='PuBuGn_d')
plt.title('Top 10')
plt.xlabel("Player")
plt.xticks(rotation=45)
plt.grid()
plt.tight_layout()
plt.show()

Correlation between physique and overall

In [ ]:
plt.figure(figsize=(10,6))
newdf =data[['weight_kg','height_cm','physic', 'overall']]
correlation = newdf.corr()
sns.heatmap(correlation, cmap="Blues",) 
plt.tight_layout()

Histograms of common numerical features

In [ ]:
dataset1 = data[['age','height_cm','weight_kg','club','overall','potential','preferred_foot',
                 'weak_foot','work_rate']]
dataset1.hist(bins='auto', figsize=(14, 10))
plt.show()

Most valuable positions

In [ ]:
data['main_position']=data['player_positions'].str.split(pat=',', n=-1, expand=True)[0]
Players_grouped=data.groupby('main_position')['value_eur'].mean()/1e6
Players_grouped=Players_grouped.sort_values()
Players_grouped.plot(kind='barh',figsize=(12,8))
plt.grid()
plt.xlabel("Average value (Million euro) ")
plt.show()

Number of players from each country

In [ ]:
counts_Nationality = data["nationality"].value_counts()
counts_Nationality = counts_Nationality.reset_index()
counts_Nationality.columns= ["Nations","Counts"]
sns.catplot(y="Nations",x="Counts",data=counts_Nationality.head(50),height=20,kind="bar")
plt.title("Nation Wise Players Counts",fontsize=25)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.grid()
plt.show()

***Specific Club and Country Information***

In [ ]:
#Function to extract specific club players
def Club(x):
    return data[data['club'] == x ][
        ['short_name','age','club',"player_positions","overall","value_eur"]].sort_values(by=['overall'],ascending=False)

#Function to extract specific country players
def Country(x):
    return data[data['nationality'] == x ][['short_name','age','nationality',"player_positions",
                                            "overall","value_eur"]].sort_values(by=['overall'],ascending=False)

In [ ]:
#Testing with some clubs and contries
Chelsea = Club("Chelsea")
PSG = Club("Paris Saint-Germain")
Juventus = Club("Juventus")
BVB = Club("Borussia Dortmund")
Germany = Country("Germany")
England = Country("England")
France = Country("France")

In [ ]:
#Visualization of specific club and country players
plt.figure(figsize=(30, 12))
plt.bar(Chelsea["short_name"],Chelsea["overall"],color="b")
plt.xticks(rotation=90,fontsize=20)
plt.yticks(fontsize=20)
plt.title("Chelsea players", fontsize = 25)
plt.ylabel("Overall",fontsize = 20)
plt.xlabel("Player",fontsize = 20)
plt.grid()
plt.show()

plt.figure(figsize=(30, 12))
plt.bar(Juventus["short_name"],Juventus["overall"],color="k")
plt.xticks(rotation=90,fontsize=20)
plt.yticks(fontsize=20)
plt.title("Juventus players", fontsize = 25)
plt.ylabel("Overall",fontsize = 20)
plt.xlabel("Player",fontsize = 20)
plt.grid()
plt.show()

plt.figure(figsize=(30, 12))
plt.bar(Germany["short_name"].head(25),Germany["overall"].head(25),color="r")
plt.xticks(rotation=90,fontsize=20)
plt.yticks(fontsize=20)
plt.title("Top 25 German players", fontsize = 25)
plt.ylabel("Overall",fontsize = 20)
plt.xlabel("Player",fontsize = 20)
plt.grid()
plt.show()

plt.figure(figsize=(30, 12))
plt.bar(France["short_name"].head(25),France["overall"].head(25),color="g")
plt.xticks(rotation=90,fontsize=20)
plt.yticks(fontsize=20)
plt.title("Top 25 French players", fontsize = 25)
plt.ylabel("Overall",fontsize = 20)
plt.xlabel("Player",fontsize = 20)
plt.grid()
plt.show()

**Preferred Foot**

In [ ]:
data['preferred_foot'].value_counts()

label = data['preferred_foot'].value_counts().index
name = data['preferred_foot'].value_counts()
explode = [0,0.3]
plt.pie(name,labels = label,shadow = True,autopct='%1.1f%%',explode =explode, radius = 2)
plt.title('Perecentage of the preferred foot')
plt.legend()
plt.show()